In [2]:
import xml.etree.ElementTree as ET
import duckdb
from duckdb import ConstraintException
from pathlib import Path

In [3]:
con = duckdb.connect(database='german-parliament', read_only=False)

In [4]:
def create_db(reset_db=False):
    """
    Creates the necessary tables in the DuckDB database for storing plenary minutes.
    This function creates the 'party', 'speaker', and 'plenary_minute' tables, as well as sequences for party and speaker IDs.

    Args:
        reset_db (bool): If True, drops existing tables and sequences before creating new ones.
                          Defaults to False.

    Returns:
        None
    """
    if reset_db:
        con.execute("DROP TABLE IF EXISTS speech;")
        con.execute("DROP TABLE IF EXISTS speaker;")
        con.execute("DROP TABLE IF EXISTS party;")
        con.execute("DROP TABLE IF EXISTS plenary_minute;")
        con.execute("DROP TABLE IF EXISTS session;")


        con.execute("DROP SEQUENCE IF EXISTS party_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS speaker_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS legislative_period_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS session_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS speech_id_seq;")

    # Create a sequence for party IDs to ensure unique IDs for each party. It's like an auto-incrementing primary key.
    con.execute("CREATE SEQUENCE IF NOT EXISTS party_id_seq START 1;")
    # Create the party which will be used to identify a party
    con.execute(
        """
        CREATE TABLE IF NOT EXISTS party (
            id INTEGER DEFAULT nextval('party_id_seq') PRIMARY KEY,
            -- If we insert a party we just use DEFAULT to get the next value from the sequence.
            name VARCHAR UNIQUE NOT NULL
        );
        """
    )

    # Create a sequence for speaker IDs to ensure unique IDs for each speaker.
    con.execute("CREATE SEQUENCE IF NOT EXISTS speaker_id_seq START 1;")
    # Create the speaker which will be used to identify a speaker
    con.execute("""
        CREATE TABLE IF NOT EXISTS speaker (
            id INTEGER DEFAULT nextval('speaker_id_seq') UNIQUE NOT NULL,
            -- If we insert a speaker we just use DEFAULT to get the next value from the sequence.
            name VARCHAR PRIMARY KEY NOT NULL,
            party_id INTEGER REFERENCES party(id) NOT NULL
        );
        """
    )

    # Create a sequence for legislative periods to ensure unique IDs for each legislative period.
    con.execute("CREATE SEQUENCE IF NOT EXISTS legislative_period_id_seq START 1;")
    # Create the session table to store information about legislative periods and sessions.
    con.execute("""
        CREATE TABLE IF NOT EXISTS session (
            id INTEGER DEFAULT nextval('legislative_period_id_seq') UNIQUE NOT NULL,
            legislative_period INTEGER NOT NULL,
            session_no INTEGER NOT NULL,
            date DATE NOT NULL,
            primary key (legislative_period, session_no)
        );
        """
    )

    # Create a sequence for plenary minute IDs to ensure unique IDs for each plenary minute.
    con.execute("CREATE SEQUENCE IF NOT EXISTS speech_id_seq START 1;")
    con.execute("""
        CREATE TABLE IF NOT EXISTS speech (
            id BIGINT DEFAULT nextval('speech_id_seq') PRIMARY KEY,
            session_id INTEGER REFERENCES session(id) NOT NULL,
            speaker_id INTEGER REFERENCES speaker(id) NOT NULL,
            role VARCHAR, -- Role of the speaker, e.g. 'mp'
            position VARCHAR, -- Position of the speaker, e.g. 'Präsident'
            content TEXT NOT NULL
        );
        """
    )
    con.commit()

In [11]:
def parse_xml(file_path):
    """
    Parses an XML file and returns the root element.

    Args:
        file_path (str): The path to the XML file.

    Returns:
        xml.etree.ElementTree.Element: The root element of the parsed XML tree,
                                         or None if parsing fails.
    """
    try:
        print(f"Parsing file at path: {file_path}")
        tree = ET.parse(file_path)
        root = tree.getroot()
        return root
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        return None
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

In [12]:
def insert_session(titleStmt):
    """ Inserts session information into the session table.

    Args:
        titleStmt (xml.etree.ElementTree.Element): The title statement element containing legislative period and session information.

    Raises:
        Exception: If the titleStmt is None or if the publicationStmt is not found in the XML file.

    Returns:
        None
    """
    if titleStmt is None:
            # @todo handle case where titleStmt is not found. Maybe write log to file?
            raise Exception("Title statement not found in the XML file.")
    else:

        legislative_period = titleStmt.find('legislativePeriod').text
        session = titleStmt.find('sessionNo').text
        print(f"Legislative Period: {legislative_period}, Session: {session}")
        # Get Date
        publicationStmt = meta_data.find('publicationStmt')
        if publicationStmt is None:
            # @todo handle case where publicationStmt is not found. Maybe write log to file?
            raise Exception("Publication statement not found in the XML file.")
        else:
            date = publicationStmt.find('date').text
            print(f"Publication Date: {date}")#

        # Insert session into the session table(
        try:
            sql = """
                INSERT INTO session (legislative_period, session_no, date)
                VALUES (?, ?,?);
            """

            params = (
                legislative_period,
                session,
                date
            )
            con.execute(sql, params)

            # con.execute(f"""
            #     INSERT INTO session (id, legislative_period, session_no, date)
            #     VALUES (DEFAULT, '{legislative_period}', '{session}', '{date}');
            # """)
        except ConstraintException as e:
            # If the session already exists, we can ignore the error
            pass

def insert_party(attributes):
    # Insert party into the party table if it does not exist
    try:
        sql = "INSERT INTO party (name) VALUES (?);"
        con.execute(sql, (attributes["party"],))
    except ConstraintException as e:
        # If the party already exists, we can ignore the error
        pass

def insert_speaker(attributes):
    # Insert speaker into the speaker table
    try:
        sql = """
            INSERT INTO speaker (name, party_id)
            VALUES (
                ?,                                    -- speaker’s full name
                (SELECT id FROM party WHERE name = ?) -- look-up party once, safely
            );
            """

        con.execute(sql, (attributes["name"], attributes["party"]))
    except ConstraintException as e:
        # If the speaker already exists, we can ignore the error
        pass

def insert_speech(speech, attributes):
    # Insert speech into the speech table
    speech_string = ET.tostring(speech, encoding='unicode') if speech is not None else ''
    try:
        party_id = con.execute(f"""
                        SELECT id FROM party WHERE name = '{attributes['party']}';
                    """).fetchone()[0]
        speaker_id = con.execute(f"""
                        SELECT id FROM speaker WHERE name = '{attributes['name']}' AND party_id = {party_id};
                    """).fetchone()[0]
        session_id = con.execute(f"""
                        SELECT id FROM session WHERE legislative_period = '{legislative_period}' AND session_no = '{session}';
                     """).fetchone()[0]

        sql = """
        INSERT INTO speech
               (session_id, speaker_id, role, position, content)
        VALUES (?, ?, ?, ?, ?);
        """

        # values to bind — order must match the placeholders
        params = (
            session_id,
            speaker_id,
            attributes.get("role"),      # None → NULL in SQL
            attributes.get("position"),
            speech_string
        )

        con.execute(sql, params)

    except ConstraintException as e:
        # If the speech already exists, we can ignore the error
        pass

def parse_plenary_minute(xml_root):
    """ Parses the plenary minute XML root element and extracts speeches, speakers, and parties and inserts them into the database.

    Args:
        xml_root (xml.etree.ElementTree.Element): The root element of the XML tree containing the data of a single plenary minute.

    Raises:
        Exception: If the XML root element is None.

    Returns:
        None

    """
    if xml_root is not None:
        meta_data = root.find('teiHeader').find('fileDesc')
        # Get legislative period and session
        titleStmt = meta_data.find('titleStmt')
        insert_session(titleStmt)

        plenary_minute = root.find('text').find('body').findall('div')
        speeches = []
        # Get all 'div' elements in the 'body' of the 'text'. Those are the elements that contain the actual plenary minute content.
        for div in plenary_minute:
            speeches.extend(div.findall('sp'))

        for speech in speeches:
            # Get attributes of the speech element
            attributes = speech.attrib
            insert_party(attributes)
            insert_speaker(attributes)
            insert_speech(speech, attributes)
    else:
        raise Exception("Missing XML root element.")

def parse_legislative_period(path):
    print(f"Parsing legislative period at path: {path}")
    for file in Path(path).rglob("*"):
        if file.suffix == ".xml":
            root = parse_xml(file)
            parse_plenary_minute(root)


def main():
    # Example usage:
    # file_to_parse = '../data/plenary_minutes/wahlperiode_01/BT_01_029.xml'
    # create_db(True)
    # root = parse_xml(file_to_parse) #This is the 'TEI' element
    # parse_plenary_minute(root)
    create_db(True)
    for period in range(1, 2):
        # 02:d adjusts the number to be two digits, e.g. 01, 02, ..., 10, 11, ...
        parse_legislative_period(f'../data/plenary_minutes/wahlperiode_{period:02d}')

if __name__ == "__main__":
    main()

Parsing legislative period at path: ../data/plenary_minutes/wahlperiode_01


In [ ]:

    # for speech in speeches:
    #     # Get attributes of the speech element
    #     attributes = speech.attrib
    #     # Insert party into the party table if it does not exist
    #     try:
    #         con.execute(f"INSERT INTO party (id, name) VALUES (DEFAULT, '{attributes['party']}');")
    #     except ConstraintException as e:
    #         # If the party already exists, we can ignore the error
    #         pass
    #     # Insert speaker into the speaker table
    #     try:
    #         con.execute(f"""
    #             INSERT INTO speaker (id, name, party_id)
    #             VALUES (DEFAULT, '{attributes['name']}',
    #                     (SELECT id FROM party WHERE name = '{attributes['party']}'));
    #         """)
    #     except ConstraintException as e:
    #         # If the speaker already exists, we can ignore the error
    #         pass
    #     # Insert speech into the speech table
    #     speech_string = ET.tostring(speech, encoding='unicode') if speech is not None else ''
    #     try:
    #         party_id = con.execute(f"""
    #                         SELECT id FROM party WHERE name = '{attributes['party']}';
    #                     """).fetchone()[0]
    #         speaker_id = con.execute(f"""
    #                         SELECT id FROM speaker WHERE name = '{attributes['name']}' AND party_id = {party_id};
    #                     """).fetchone()[0]
    #         session_id = con.execute(f"""
    #                         SELECT id FROM session WHERE legislative_period = '{legislative_period}' AND session_no = '{session}';
    #                      """).fetchone()[0]
    #
    #         sql = """
    #         INSERT INTO speech
    #                (session_id, speaker_id, role, position, content)
    #         VALUES (?, ?, ?, ?, ?);
    #         """
    #
    #         # values to bind — order must match the placeholders
    #         params = (
    #             session_id,
    #             speaker_id,
    #             attributes.get("role"),      # None → NULL in SQL
    #             attributes.get("position"),
    #             speech_string
    #         )
    #
    #         con.execute(sql, params)
    #
    #     except ConstraintException as e:
    #         # If the speech already exists, we can ignore the error
    #         pass
    #
    #     # print(ET.tostring(speech, encoding='unicode').strip(), "\n\n")
    #     # speech_text = ''.join([p.text for p in speech_content if p.text is not None])


In [9]:
con.execute("select speech.id, speech.speaker_id, role, position, speaker.name, content from speech join speaker on speech.speaker_id = speaker.id;").fetchdf()

,id,speaker_id,role,position,name,content


In [140]:
con.execute("select * from speaker join party on speaker.party_id = party.id ;").fetchdf()

,id,name,party_id,id_1,name_1


In [157]:
con.execute("select * from session;").fetchdf()

,id,legislative_period,session_no,date
0,1,1,29,1950-01-20
